In [1]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style('whitegrid')
import h5py
import os
from tqdm import tqdm_notebook as tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/home/xupeng/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from scipy.io import loadmat

In [3]:
data = loadmat('/home/xupeng/projects/signal/data/citydata.mat')

In [5]:
data['RX']

array([[5.976480e+05, 4.133011e+06, 2.301000e+01],
       [5.976580e+05, 4.132991e+06, 2.301000e+01],
       [5.976680e+05, 4.132981e+06, 2.301000e+01],
       ...,
       [5.995080e+05, 4.132411e+06, 2.601000e+01],
       [5.995080e+05, 4.132421e+06, 2.601000e+01],
       [5.995180e+05, 4.132421e+06, 2.601000e+01]])

In [6]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'RX', 'RSS', 'TOA', 'RSS_t', 'TX', 'DOA'])

In [ ]:
with h5py.File('/home/xupeng/projects/signal/data/citydata.h5') as f:
    f.create_dataset('RX',data = data['RX'])

In [8]:
a = np.random.normal(0,1,10)
b = np.random.normal(0,1,10)

In [9]:
a,b

(array([-0.18420154, -1.13120216,  1.70978614,  0.01923934, -0.09253187,
        -0.8967794 , -2.32134271,  1.50332926,  0.74150266,  2.36464928]),
 array([-0.35385913, -1.4250532 ,  0.45390619,  0.80401127,  0.30080825,
         0.40323843, -0.04233569, -0.90435086,  2.62585876, -0.67848633]))

In [11]:
np.corrcoef(a,b)[0,1]

0.06618708636836773

In [12]:
from scipy.stats import pearsonr

In [14]:
pearsonr(a,b)[0]

0.06618708636836772

In [15]:
import numpy as np
def pcc(X, Y):
    X -= X.mean(0)
    Y -= Y.mean(0)
    X /= X.std(0)
    Y /= Y.std(0)
    return np.mean(X*Y)

pcc(a,b)

0.0661870863683677

In [1]:
cd ..

/home/xupeng/projects/signal


In [3]:
import gc, argparse, sys, os, errno
sys.path.append('bin')

In [4]:

import numpy as np
from tqdm import tqdm
import scipy
from scipy.stats import pearsonr
from scipy.io import loadmat
from matplotlib.mlab import griddata
tableau20 = np.array([(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)])/255.
styles = ["white","dark",'whitegrid',"darkgrid"]
contexts = ['paper','talk','poster','notebook']
from ipywidgets import interact, FloatSlider,IntSlider, RadioButtons,Dropdown,Tab,Text,SelectMultiple,FloatRangeSlider
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from mpl_toolkits.mplot3d.axes3d import *
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler,Imputer
from sklearn.feature_selection import VarianceThreshold

from utils import prepare_dataset,preprocess_zscore, get_original_parameters, report_metrics, mdn_logp, mdn_loss
from model import MixtureDensityNetwork, IsotropicGaussianMixture
import argparse
import numba as nb

In [5]:
feature_counts = 10
point_counts = 1000
test_counts = 100
rawdata = loadmat('/home/xupeng/projects/signal/data/citydata.mat')
RSS = rawdata['RSS']
TOA = rawdata['TOA']
DOA = rawdata['DOA']
TXloc = rawdata['TX']
RXloc = rawdata['RX']
xyzorigin_RX = np.array([np.min(RXloc[:,0]),np.min(RXloc[:,1]),0])
xyzorigin_TX = np.array([np.min(TXloc[:,0]),np.min(TXloc[:,1]),0])
RXloc_shifted = RXloc - xyzorigin_RX
TXloc_shifted = TXloc - xyzorigin_TX

In [ ]:
distance=np.zeros((1500,1500))
for i in tqdm(range(1500)):
    for j in range(1500):
        distance[i,j]=np.linalg.norm(RXloc_shifted[i]-RXloc_shifted[j])

In [20]:
from numba import cuda



In [39]:


c=np.append(RSS,TOA,axis=1)
combine_data=np.append(c,DOA,axis=1)
@nb.jit()
def pcc(X, Y):
    X = X - np.mean(X)
    Y = Y - np.mean(Y)
    X = X/X.std(0)
    Y = Y/Y.std(0)
    return np.mean(X*Y)
#index_1 index_2 should be two of the 1500 receiver
@nb.jit()
def relate(index_1, index_2, features):
    xdata=combine_data[index_1]
    xdata=xdata[features]
    ydata=combine_data[index_2]
    ydata=ydata[features]
    an=~np.isnan(xdata)&~np.isnan(ydata)
    xdata=xdata[an]
    nr=1-len(xdata)/len(ydata)
    ydata=ydata[an]
    an=an[an]
    r=pcc(xdata,ydata)
    return r,nr
@nb.jit()
def corelate(features):
    r=np.zeros(testnum)
    nr=np.zeros(testnum)
    xn=np.random.randint(1,1500,size=(testnum,batchsize))
    yn=np.random.randint(1,1500,size=(testnum,batchsize))
    r=np.zeros((testnum,batchsize))
    u=np.zeros((testnum,batchsize))
    d=np.zeros((testnum,batchsize))
    re=np.zeros(testnum)
    nr=np.zeros(testnum)
    for j in range(testnum):
        nan_num=0;
        for i in range(batchsize):
            d[j][i]=distance[xn[j][i],yn[j][i]]
            r[j][i],u[j][i]=relate(xn[j][i],yn[j][i],features)
            while np.isnan(r[j][i]):
                xn[j][i]=np.random.randint(1,1500)
                yn[j][i]=np.random.randint(1,1500)
                d[j][i]=distance[xn[j][i],yn[j][i]]
                r[j][i],u[j][i]=relate(xn[j][i],yn[j][i],features)
                nan_num=nan_num+1
        re[j]=pcc(d[j],r[j])
        nr[j]=(np.sum(u[j])+nan_num)/(nan_num+batchsize)
    return -np.mean(r), np.mean(nr), xn.flatten(), yn.flatten(), r.flatten(), u.flatten()

batchsize=100;
testnum=int(point_counts/batchsize)
x_total=np.zeros((test_counts,testnum*batchsize))
y_total=np.zeros((test_counts,testnum*batchsize))
r_total=np.zeros((test_counts,testnum*batchsize))
u_total=np.zeros((test_counts,testnum*batchsize))
relation=np.zeros(test_counts)

label=np.random.randint(1,500,size=(test_counts,feature_counts))

nan_rate=np.zeros(test_counts)

In [40]:
for i in tqdm(range(test_counts)):
    relation[i],nan_rate[i],x_total[i],y_total[i],r_total[i],u_total[i]=corelate(label[i])



  0%|          | 0/100 [00:00<?, ?it/s]/home/xupeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
  1%|          | 1/100 [00:01<02:27,  1.49s/it]/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value enco

In [ ]:


c=np.append(RSS,TOA,axis=1)
combine_data=np.append(c,DOA,axis=1)
def pcc(X, Y):
    X = X - np.mean(X)
    Y = Y - np.mean(Y)
    X = X/X.std(0)
    Y = Y/Y.std(0)
    return np.mean(X*Y)
#index_1 index_2 should be two of the 1500 receiver
def relate(index_1, index_2, features):
    xdata=combine_data[index_1]
    xdata=xdata[features]
    ydata=combine_data[index_2]
    ydata=ydata[features]
    an=~np.isnan(xdata)&~np.isnan(ydata)
    xdata=xdata[an]
    nr=1-len(xdata)/len(ydata)
    ydata=ydata[an]
    an=an[an]
    r=pcc(xdata,ydata)
    return r,nr
def corelate(features):
    r=np.zeros(testnum)
    nr=np.zeros(testnum)
    xn=np.random.randint(1,1500,size=(testnum,batchsize))
    yn=np.random.randint(1,1500,size=(testnum,batchsize))
    r=np.zeros((testnum,batchsize))
    u=np.zeros((testnum,batchsize))
    d=np.zeros((testnum,batchsize))
    re=np.zeros(testnum)
    nr=np.zeros(testnum)
    for j in range(testnum):
        nan_num=0;
        for i in range(batchsize):
            d[j][i]=distance[xn[j][i],yn[j][i]]
            r[j][i],u[j][i]=relate(xn[j][i],yn[j][i],features)
            while np.isnan(r[j][i]):
                xn[j][i]=np.random.randint(1,1500)
                yn[j][i]=np.random.randint(1,1500)
                d[j][i]=distance[xn[j][i],yn[j][i]]
                r[j][i],u[j][i]=relate(xn[j][i],yn[j][i],features)
                nan_num=nan_num+1
        re[j]=pcc(d[j],r[j])
        nr[j]=(np.sum(u[j])+nan_num)/(nan_num+batchsize)
    return -np.mean(r), np.mean(nr), xn.flatten(), yn.flatten(), r.flatten(), u.flatten()

batchsize=100;
testnum=int(point_counts/batchsize)
x_total=np.zeros((test_counts,testnum*batchsize))
y_total=np.zeros((test_counts,testnum*batchsize))
r_total=np.zeros((test_counts,testnum*batchsize))
u_total=np.zeros((test_counts,testnum*batchsize))
relation=np.zeros(test_counts)

label=np.random.randint(1,500,size=(test_counts,feature_counts))

nan_rate=np.zeros(test_counts)

In [41]:
for i in tqdm(range(test_counts)):
    relation[i],nan_rate[i],x_total[i],y_total[i],r_total[i],u_total[i]=corelate(label[i])



  0%|          | 0/100 [00:00<?, ?it/s]/home/xupeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.typ